In [5]:
import pandas as pd
import numpy as np
import random


df = pd.read_csv('~/Documents/MooreLab/breastCancer29.csv', sep=',', dtype=np.float64)

random.seed(42)
list = random.sample(range(1,100000), 2504)


#2504 total items
extra_col = pd.Series(list)

df.insert(0,'patient_id', extra_col.values)
df_id = df.drop('phenotype', axis=1)



In [171]:
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split


df2 = pd.read_csv('finalBCdata.csv')
df2_id = df2.drop('phenotype', axis=1)
features = df2_id.drop('patient_id', axis=1).values

training_features, testing_features, training_target, testing_target = \
            train_test_split(features, df2['phenotype'].values, random_state=42)

exported_pipeline = ExtraTreesClassifier(bootstrap=False, random_state = 42, criterion="entropy", max_features=0.1, min_samples_leaf=18, min_samples_split=20, n_estimators=100)
exported_pipeline.fit(training_features, training_target)



SNP_names = df2.columns.values.tolist()[1:-1]
n_snps = len(SNP_names)

all_info = []
x_row = df2_id.loc[df2_id.patient_id == 1221, :]
x_features = x_row.drop('patient_id', axis=1).values
x_pos = exported_pipeline.predict_proba(x_features)[:, 1]

mylistup = []
mylistdown = []

for snp in range(0, x_features.size):
    x_features[0][snp] = (x_features[0][snp] + 1) % 3
    probup = exported_pipeline.predict_proba(x_features[0].reshape(1, -1))[0, 1]
    mylistup.append(probup)
    
    x_features[0][snp] = (x_features[0][snp] - 2) % 3  # -2+1=-1
    probdown = exported_pipeline.predict_proba(x_features.reshape(1, -1))[0, 1]
    mylistdown.append(probdown)

    x_features[0][snp] = (x_features[0][snp] + 1) % 3  # change back
    
upbumplist = mylistup - original_prob
downbumplist = mylistdown - original_prob
uplist = upbumplist.tolist()
downlist = downbumplist.tolist()

completelist = uplist + downlist + [0]  # 0 for no change
min_index = np.argmin(completelist)
min_value = min(completelist)
best_prob = min_value + original_prob
all_info.append(str(1221))  # participant idx

all_info.extend((original_prob, best_prob, original_prob - best_prob))
if (min_index == (n_snps * 2 + 1)):  # no change is best
    all_info.extend(("N/A", "N/A", "N/A"))
else:
    SNP_idx = min_index % n_snps
    all_info.extend((SNP_names[SNP_idx], int(x_features[0][SNP_idx]),
                     int((x_features[0][SNP_idx] - 1 + 2 * float(min_index <= n_snps)) % 3)))

final_df = pd.DataFrame(all_info).T
final_df.columns = ['Participant Idx', 'Original Prob.', 'Best Possible Prob.', 'Risk Decrease', 'SNP Name', 'Ori. SNP Value', 'New SNP Value']
final_df






,Participant Idx,Original Prob.,Best Possible Prob.,Risk Decrease,SNP Name,Ori. SNP Value,New SNP Value
0,1221,0.399632,0.374337,0.0252943,rs1011970,0,2


In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split

df = pd.read_csv('finalBCdata.csv')
df_id = df.drop('phenotype', axis=1)
features = df_id.drop('patient_id', axis=1).values

training_features, testing_features, training_target, testing_target = \
            train_test_split(features, df['phenotype'].values, random_state=42)

exported_pipeline = ExtraTreesClassifier(bootstrap=False, random_state = 42, criterion="entropy", max_features=0.1, min_samples_leaf=18, min_samples_split=20, n_estimators=100)
exported_pipeline.fit(training_features, training_target)


# def individual_editing(sub_id, SNP, new_SNP_val):
all_info = []
x_row = df_id.loc[df_id.patient_id == 1221, :]
x_features = x_row.drop('patient_id', axis=1).values
original_prob = exported_pipeline.predict_proba(x_features)[:, 1][0]

x_row.loc[:, 'rs6762644'].values[0] = 1
new_x_features = x_row.drop('patient_id', axis=1).values
new_prob = exported_pipeline.predict_proba(new_x_features)[:, 1][0]
print(original_prob)
print(new_prob)



0.39963159229455625
0.39793218105095024


In [2]:
df

,patient_id,rs616488,rs11249433,rs4849887,rs2016394,rs1550623,rs6762644,rs4973768,rs1053338,rs1353747,...,rs3817198,rs17356907,rs1292011,rs999737,rs11627032,rs13329835,rs1436904,rs3760982,rs2823093,phenotype
0,83811,1.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,...,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0
1,14593,1.0,1.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,...,0.0,1.0,1.0,2.0,2.0,0.0,1.0,1.0,0.0,1.0
2,3279,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
3,97197,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,1.0,2.0,2.0,1.0,0.0,0.0,0.0,2.0,1.0,0.0
4,36049,1.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.0,1.0,1.0,2.0,0.0,1.0
5,32099,0.0,0.0,1.0,1.0,1.0,2.0,2.0,0.0,1.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
6,29257,1.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
7,18290,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,1.0
8,96531,1.0,1.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0
9,13435,0.0,1.0,1.0,2.0,0.0,2.0,2.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0


In [2]:
df = pd.read_csv('finalBCdata.csv')
value = df.loc[df.patient_id == 1221, 'rs6762644']
value.values

array([2.])

In [18]:
import plotly.figure_factory as ff
import pandas as pd
import numpy as np
import io
import requests

url = "https://raw.githubusercontent.com/Hoytgong/vgmed/master/finalBCdata.csv"
s = requests.get(url).content
df = pd.read_csv(io.StringIO(s.decode('utf-8')))

df['patient_id'].values.tolist()



[83811,
 14593,
 3279,
 97197,
 36049,
 32099,
 29257,
 18290,
 96531,
 13435,
 88697,
 97081,
 71483,
 11396,
 77398,
 55303,
 4166,
 3906,
 12281,
 28658,
 30496,
 66238,
 78908,
 3479,
 73564,
 26063,
 93851,
 85182,
 91925,
 71427,
 54988,
 28894,
 58879,
 77237,
 36464,
 852,
 99459,
 20927,
 91507,
 55393,
 44598,
 36422,
 20380,
 28222,
 44119,
 13397,
 12157,
 49798,
 12677,
 47053,
 45083,
 79132,
 34672,
 5696,
 95648,
 60218,
 70285,
 16362,
 49616,
 10329,
 72358,
 38428,
 82398,
 81071,
 47401,
 75675,
 25204,
 92350,
 9117,
 6007,
 86674,
 29872,
 37931,
 10459,
 30513,
 13239,
 49824,
 36435,
 59430,
 83321,
 47820,
 21320,
 48521,
 46567,
 27461,
 87842,
 34994,
 91989,
 89594,
 84940,
 9359,
 79841,
 83228,
 22432,
 70011,
 95569,
 32088,
 21418,
 60590,
 49736,
 35383,
 83887,
 90199,
 73001,
 28786,
 89734,
 42505,
 7332,
 30022,
 4208,
 41348,
 52582,
 35094,
 8676,
 27654,
 74342,
 94099,
 41246,
 27870,
 85910,
 65436,
 51857,
 84260,
 60143,
 18727,
 34719,
 1830